In [ ]:
!pip install tensorflow
!pip install tflite-runtime
!pip install numpy

In [ ]:
import tensorflow as tf
from tensorflow import keras

import tflite_runtime.interpreter as tflite

import numpy as np

In [ ]:
# Load the cifar10 dataset
cifar10 = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

print(train_images.shape)

In [ ]:
input_layer = keras.layers.Input(shape=(32,32,3), name="InputLayer", dtype='float32')

hidden_layer = keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name="Layer0")(input_layer)
hidden_layer = keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2), padding='valid', name="Layer1")(hidden_layer)

hidden_layer = keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name="Layer2")(hidden_layer)
hidden_layer = keras.layers.MaxPool2D(pool_size=(2, 2),strides=(2,2), padding='same', name="Layer3")(hidden_layer)

hidden_layer = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name="Layer4")(hidden_layer)
hidden_layer = keras.layers.MaxPool2D(pool_size=(2, 2),strides=(2,2), padding='same', name="Layer5")(hidden_layer)

hidden_layer = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name="Layer6")(hidden_layer)
hidden_layer = keras.layers.MaxPool2D(pool_size=(2, 2),strides=(2,2), padding='same', name="Layer7")(hidden_layer)

hidden_layer = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name="Layer8")(hidden_layer)
hidden_layer = keras.layers.MaxPool2D(pool_size=(2, 2),strides=(2,2), padding='same', name="Layer9")(hidden_layer)

hidden_layer = keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name="Layer10")(hidden_layer)
hidden_layer = keras.layers.MaxPool2D(pool_size=(2, 2),strides=(1,1), padding='same', name="Layer11")(hidden_layer)

hidden_layer = keras.layers.Conv2D(filters=1024, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name="Layer12")(hidden_layer)

route13 = hidden_layer

hidden_layer = keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding='same', name="Layer13")(hidden_layer)
hidden_layer = keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name="Layer14")(hidden_layer)
hidden_layer = keras.layers.Conv2D(filters=255, kernel_size=(1,1), strides=(1,1), activation='linear', padding='same', name="Layer15")(hidden_layer)

hidden_layer = keras.layers.Concatenate(name="Layer17")([hidden_layer, route13])

hidden_layer = keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), activation='relu', padding='same', name="Layer18")(hidden_layer)

hidden_layer = keras.layers.Conv2DTranspose(size=(2, 2), name="Layer19")(hidden_layer)
route19 = hidden_layer
hidden_layer = keras.layers.Concatenate(name="Layer20")([hidden_layer, route19])
hidden_layer = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name="Layer21")(hidden_layer)
hidden_layer = keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=(1,1), activation='relu', padding='same', name="Layer22")(hidden_layer)

hidden_layer = keras.layers.Flatten(name="TestLayer1")(hidden_layer)
hidden_layer = keras.layers.Dense(10, name="TestLayer2", activation="softmax")(hidden_layer)

output_layer = hidden_layer

model = keras.Model(inputs=[input_layer], outputs=[output_layer], name="Yolo")
model.summary()

In [ ]:
from tensorflow.keras import datasets, layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
# Compiliert das Modell
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
# Trainiert das Modell
model.fit(train_images[0:100], train_labels[0:100], epochs=1, validation_data=(test_images[0:100], test_labels[0:100]))

In [ ]:
import matplotlib.pylab as plt
plt.imshow(train_images[0], cmap=plt.cm.binary)
print(test_labels[0])
plt.show()
output = model.predict(train_images[0:1])
print(np.argmax(output))

### Generiert den "representativen Datensatz" für die Quantifizierung

In [ ]:
def representative_data_gen():
    dataset = tf.data.Dataset.from_tensor_slices(train_images).batch(1)

    for input_value in dataset.take(100):
        yield [input_value]

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

with open('tinyYOLO.tflite', 'wb') as f:
  f.write(tflite_quant_model)

### Durchführung der Post-Integer-Quantifizierung

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

with open('tinyYOLO.tflite', 'wb') as f:
  f.write(tflite_quant_model)

### Compilierung des Tensorflow-Lite-Modells für die Edge TPU

In [ ]:
! edgetpu_compiler -s tinyYOLO.tflite

### Nutzung des Tensorflow-Lite-Modells (auf CPU)

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="tinyYOLO.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(test_images[100:101] * 255, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(np.argmax(output_data))



class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

import matplotlib.pylab as plt
plt.imshow(test_images[100], cmap=plt.cm.binary)
print(test_labels[100])
plt.show()

### Nutzung des Modells auf der Edge TPU

In [ ]:
import platform
import time

EDGETPU_SHARED_LIB = {
  'Linux': 'libedgetpu.so.1',
  'Darwin': 'libedgetpu.1.dylib',
  'Windows': 'edgetpu.dll'
}[platform.system()]

def make_interpreter(model_file):
    model_file, *device = model_file.split('@')
    
    return tflite.Interpreter(model_path=model_file, experimental_delegates=[
          tflite.load_delegate(EDGETPU_SHARED_LIB, {'device': device[0]} if device else {})])


# Läd das Model
interpreter = make_interpreter("tinyYOLO_edgetpu.tflite")

interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']

input_data = np.array(test_images[0:1] * 255, dtype=np.int8)

interpreter.set_tensor(input_details[0]['index'], input_data) 

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
for _ in range(10000):
    start = time.perf_counter()
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    inference_time = time.perf_counter() - start
    print('%.1fms' % (inference_time * 1000))